In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-pqq5xo0q
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-pqq5xo0q
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=50e2a554efedaee1e0bcfee4cb4c2918e98b68e81064ce426c44cf513bce37b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-r6w0fe7h/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
%%writefile breadthfirst.cu
#include <iostream>
#include <queue>
#include <vector>
#include <omp.h>
using namespace std;
int main() {
    int num_vertices, num_edges, source;
    cout << "Enter number of vertices, edges, and source node: ";
    cin >> num_vertices >> num_edges >> source;
    // Input validation
    if (source < 1 || source > num_vertices) {
        cout << "Invalid source node!" << endl;
        return 1;
    }
     vector<vector<int>> adj_list(num_vertices + 1);
    for (int i = 0; i < num_edges; i++) {
        int u, v;
        cin >> u >> v;
        // Input validation for edges
        if (u < 1 || u > num_vertices || v < 1 || v > num_vertices) {
            cout << "Invalid edge: " << u << " " << v << endl;
            return 1;
        }
        adj_list[u].push_back(v);
        adj_list[v].push_back(u);
    }
    queue<int> q;
    vector<bool> visited(num_vertices + 1, false);
    q.push(source);
    visited[source] = true;
    while (!q.empty()) {
        int curr_vertex = q.front();
        q.pop();
        cout << curr_vertex << " ";
        // Sequential loop for neighbors
        for (int i = 0; i < adj_list[curr_vertex].size(); i++) {
            int neighbour = adj_list[curr_vertex][i];
            if (!visited[neighbour]) {
                visited[neighbour] = true;
                q.push(neighbour);
            }
        }
    }
    cout << endl;
    return 0;
}

Writing breadthfirst.cu


In [7]:
!nvcc breadthfirst.cu -o breadthfirst

In [8]:
!./breadthfirst

Enter number of vertices, edges, and source node: ^C


In [13]:
%%writefile depthfirst.cu
#include <iostream>
#include <vector>
using namespace std;

const int MAXN = 1e5;
vector<int> adj[MAXN+5]; // adjacency list
bool visited[MAXN+5]; // mark visited nodes

void dfs(int node) {
    visited[node] = true;
    cout << node << " "; // Print the visited node here
    for (int i = 0; i < adj[node].size(); i++) {
        int next_node = adj[node][i];
        if (!visited[next_node]) {
            dfs(next_node);
        }
    }
}

int main() {
    cout << "Please enter the number of nodes and edges: ";
    int n, m; // number of nodes and edges
    cin >> n >> m;

    cout << "Please enter the edges: \n";
    for (int i = 0; i < m; ++i) {
        int u, v;
        cin >> u >> v; // edge from u to v
        adj[u].push_back(v); // adding edge to adjacency list
        adj[v].push_back(u); // assuming an undirected graph
    }

    // Assuming the starting node for DFS is 1
    int start_node = 1;
    cout << "DFS traversal starting from node " << start_node << ": ";
    dfs(start_node);
    cout << endl;

    return 0;
}


Overwriting depthfirst.cu


In [14]:
!nvcc depthfirst.cu -o depthfirst
!./depthfirst


Please enter the number of nodes and edges: 5 7
Please enter the edges: 
1 2
1 3
1 4
2 3
3 4
3 5
4 5
DFS traversal starting from node 1: 1 2 3 4 5 
